# 6/5/24

Download ENDF and PENDF files

In [1]:
#import sys
#sys.path.append('/filespace/e/esweinstein2/ENDFtk/build')
import ENDFtk
from tendl_downloads import tendl_download
import pandas as pd

# Call function by user CLI input
element = input('Select element: ')
Z = input('Select atomic number, Z = ')
endf_path = tendl_download(element, Z, 'endf')
try:
    pendf_path = tendl_download(element, Z, 'pendf')
    print(f'ENDF file can be found at {endf_path}')
    print(f'PENDF file can be found at {pendf_path}')
except:
    # Need to figure out somewhere in the function how to
    # register a "ERROR 404: Not Found" message.
    # Probably in the download_file function
    # Resolve later
    print(f'ENDF file can be found at {endf_path}')
    print('No PENDF file found')

No Bottleneck unit testing available.
ENDF file can be found at tendl_2017_Fe056_endf.endf
PENDF file can be found at tendl_2017_Fe056_pendf.pendf


Use ENDFtk to extract data from the ENDF file

In [3]:
# Read in necessary ENDF data using ENDFtk
tape = ENDFtk.tree.Tape.from_file(endf_path)
mat_ids = tape.material_numbers
matb = mat_ids[0] 
xs_MF = 3
file = tape.material(matb).file(xs_MF)
MTs = []
# Total range of MT values from
# https://www.oecd-nea.org/dbdata/data/manual-endf/endf102_MT.pdf
for i in range(1000):
    try:
        file.section(i)
        MTs.append(i)
    except:
        continue

Create the cards for the GROUPR parameters

In [3]:
# Set Card 1
nendf = -21 # unit for endf tape
npend = -26 # unit for pendf tape
ngout1 = 0 # unit for input gout tape (default=0)
ngout2 = 31 # unit for output gout tape (default=0)

card1 = [nendf, npend, ngout1, ngout2]

# Set Card 2
# matb -- (already defined) -- material to be processed
ign = 17 # neutron group structure option
igg = 0 # gamma group structure option
iwt = 11 # weight function option
lord = 0 # Legendgre order
ntemp = 1 # number of temperatures (default = 1)
nsigz = 1 # number of sigma zeroes (default = 1)
iprint = 1 # long print option (0/1=minimum/maximum) -- (default=1)
ismooth = 1 # swith on/off smoother operation (1/0, default=1=on)

card2 = [matb, ign, igg, iwt, lord, ntemp, nsigz, iprint, ismooth]

# Set Card 3
title = f'"GROUPR Processing for {element}{Z}"'
card3 = [title]

# Set Card 4
temp = 293 # temperature in Kelvin
card4 = [temp]

# Set Card 5
sigz = 0 # sigma zero values (including infinity)
card5 = [sigz]

# Set Card 9
mt_table = pd.read_csv('./mt_table.csv')
mfd = 3 # file to be processed
mtd = MTs # sections to be processed
card9 = []
for mt in MTs:
    mtname = mt_table[mt_table['MT'] == mt]['Reaction'].values[0] # description of section to be processed
    card9_line = f'{mfd} {mt} "{mtname}"/'
    card9.append(card9_line)

# Set Card 10
matd = 0 # next mat number to be processed
card10 = [matd]

# Create a card deck
deck = [card1, card2, card3, card4, card5, card9, card10]
deck_names = ['Card 1', 'Card 2', 'Card 3', 'Card 4', 'Card 5', 'Card 9', 'Card 10']
deck_df = pd.DataFrame({
    'Card' : deck_names,
    'Contents' : deck
})

Write a file to run GROUPR using the deck of cards

In [38]:
# Write the input deck to the groupr.inp file
with open('groupr.inp', 'w') as f:
    f.write('groupr / \n! \n')
    for card_name, card_content in zip(deck_df['Card'], deck_df['Contents']):
        if card_name != 'Card 9':
            f.write(f'! {card_name} \n')
            f.write(' '.join(map(str, card_content)) + ' /\n')
            f.write('! \n')
        else:
            f.write(f'! {card_name} \n')
            for line in card_content:
                f.write(line + '\n')
            f.write('! \n')

Write out ENDF and PENDF path names to an NJOY bash script template and run the bash script

In [29]:
import subprocess

# Read the template file
with open('run_njoy_template.sh', 'r') as f:
    script_content = f.read()

# Replace placeholders with actual file paths
script_content = script_content.replace('<ENDF_PATH>', endf_path)
script_content = script_content.replace('<PENDF_PATH>', pendf_path)

# Write modified script content to run_njoy.sh file
with open('run_njoy.sh', 'w') as f:
    f.write(script_content)

# Make the script executable
subprocess.run(["chmod", "+x", "run_njoy.sh"])

# Execute the modified script
njoy_run_message = subprocess.run(["./run_njoy.sh"], capture_output=True, text=True)
print(njoy_run_message.stdout)
print(njoy_run_message.stderr)

Running NJOY...
NJOY encountered an error. Check groupr.out for details.

At line 1016 of file /filespace/e/esweinstein2/NJOY2016/src/groupr.f90 (unit = 5, file = 'stdin')
Fortran runtime error: Bad integer for item 1 in list input

Error termination. Backtrace:
#0  0x7fa0a91e7bd0 in ???
#1  0x7fa0a91e8685 in ???
#2  0x7fa0a91e925b in ???
#3  0x7fa0a9414048 in ???
#4  0x7fa0a94170e1 in ???
#5  0x7fa0a9417cd1 in ???
#6  0x7fa0a9626f17 in ???
#7  0x7fa0a9628f61 in ???
#8  0x55da7e23f764 in ???
#9  0x55da7e23f2ce in ???
#10  0x7fa0a9017d09 in __libc_start_main
	at ../csu/libc-start.c:308
#11  0x55da7e23f309 in ???
#12  0xffffffffffffffff in ???



# 6/6/24 Testing

In [95]:
import ENDFtk
from tendl_downloads import tendl_download
import pandas as pd

# Call function by user CLI input
element = input('Select element: ')
Z = input('Select atomic number, Z = ')
endf_path = tendl_download(element, Z, 'endf')
try:
    pendf_path = tendl_download(element, Z, 'pendf')
    print(f'ENDF file can be found at {endf_path}')
    print(f'PENDF file can be found at {pendf_path}')
except:
    # Need to figure out somewhere in the function how to
    # register a "ERROR 404: Not Found" message.
    # Probably in the download_file function
    # Resolve later
    print(f'ENDF file can be found at {endf_path}')
    print('No PENDF file found')

ENDF file can be found at tape20
PENDF file can be found at tape21


In [96]:
# Read in necessary ENDF data using ENDFtk
tape = ENDFtk.tree.Tape.from_file(endf_path)
mat_ids = tape.material_numbers
matb = mat_ids[0] 
xs_MF = 3
file = tape.material(matb).file(xs_MF)
MTs = []
# Total range of MT values from
# https://www.oecd-nea.org/dbdata/data/manual-endf/endf102_MT.pdf
for i in range(1000):
    try:
        file.section(i)
        MTs.append(i)
    except:
        continue

In [97]:
# Set Card 1
nendf = 20 # unit for endf tape
npend = 21 # unit for pendf tape
ngout1 = 0 # unit for input gout tape (default=0)
ngout2 = 31 # unit for output gout tape (default=0)

card1 = [nendf, npend, ngout1, ngout2]

# Set Card 2
# matb -- (already defined) -- material to be processed
ign = 17 # neutron group structure option
igg = 0 # gamma group structure option
iwt = 11 # weight function option
lord = 0 # Legendgre order
ntemp = 1 # number of temperatures (default = 1)
nsigz = 1 # number of sigma zeroes (default = 1)
iprint = 1 # long print option (0/1=minimum/maximum) -- (default=1)
#ismooth = 1 # swith on/off smoother operation (1/0, default=1=on)

card2 = [matb, ign, igg, iwt, lord, ntemp, nsigz, iprint]

# Set Card 3
title = f'"GROUPR Processing for {element}{Z}"'
card3 = [title]

# Set Card 4
temp = 293.16 # temperature in Kelvin
card4 = [temp]

# Set Card 5
sigz = 0 # sigma zero values (including infinity)
card5 = [sigz]

# Set Card 9
mt_table = pd.read_csv('./mt_table.csv')
mfd = 3 # file to be processed
mtd = MTs # sections to be processed
card9 = []
for mt in MTs:
    mtname = mt_table[mt_table['MT'] == mt]['Reaction'].values[0] # description of section to be processed
    card9_line = f'{mfd} {mt} "{mtname}"'
    card9.append(card9_line)

# Set Card 10
matd = 0 # next mat number to be processed
card10 = [matd]

# Create a card deck
deck = [card1, card2, card3, card4, card5, card9, card10]
deck_names = ['Card 1', 'Card 2', 'Card 3', 'Card 4', 'Card 5', 'Card 9', 'Card 10']
deck_df = pd.DataFrame({
    'Card' : deck_names,
    'Contents' : deck
})

IndexError: index 0 is out of bounds for axis 0 with size 0

In [25]:
# Function to fomrat the cards properly
def format_card(card_name, card_content):
    card_str = ''
    gen_str = ' ' + ' '.join(map(str, card_content))
    if card_name == 'Card 9':
        for line in card_content:
            card_str += f' {line}/\n'
    elif card_name == 'Card 4':
        card_str += gen_str + '\n'
    else:
        card_str += gen_str + '/\n'
    return card_str

# Write the input deck to the groupr.inp file
with open('groupr.inp', 'w') as f:
    f.write('groupr\n')
    for card_name, card_content in zip(deck_names, deck):
        f.write(format_card(card_name, card_content))
    f.write(' 0/\nstop')

In [26]:
import subprocess

# Read the template file
with open('run_njoy_template.sh', 'r') as f:
    script_content = f.read()

# Replace placeholders with actual file paths
script_content = script_content.replace('<ENDF_PATH>', endf_path)
script_content = script_content.replace('<PENDF_PATH>', pendf_path)

# Write modified script content to run_njoy.sh file
with open('run_njoy.sh', 'w') as f:
    f.write(script_content)

# Make the script executable
subprocess.run(["chmod", "+x", "run_njoy.sh"])

# Execute the modified script
njoy_run_message = subprocess.run(["./run_njoy.sh"], capture_output=True, text=True)
print(njoy_run_message.stdout)
print(njoy_run_message.stderr)

Running NJOY...
NJOY ran successfully. Output written to groupr.out.




In [37]:
njoy_run_message.stderr == ''

True

# Converting to a script form

In [94]:
# Import packages
import ENDFtk
from tendl_downloads import tendl_download
import subprocess
import pandas as pd

# Define functions

# Function to extract MT and MAT data from an ENDF file
def endf_specs(endf_path):
    # Read in ENDF tape using ENDFtk
    tape = ENDFtk.tree.Tape.from_file(endf_path)

    # Determing the material ID
    mat_ids = tape.material_numbers
    matb = mat_ids[0]

    # Set MF for cross sections
    xs_MF = 3

    # Extract out the file
    file = tape.material(matb).file(xs_MF)

    # Extract the MT numbers that are present in the file
    MTs = []
    for i in range(1000):
        try:
            file.section(i)
            MTs.append(i)
        except:
            continue

    return matb, MTs

# Function to fomrat GROUPR input cards
def format_card(card_name, card_content):
    card_str = ''
    gen_str = ' ' + ' '.join(map(str, card_content))
    if card_name == 'Card 9':
        for line in card_content:
            card_str += f' {line}/\n'
    elif card_name == 'Card 4':
        card_str += gen_str + '\n'
    else:
        card_str += gen_str + '/\n'
    return card_str

# Function to create the GROUPR input file
def groupr_input(matb, mt_table):
    
    # INPUT PARAMETERS

    # Set Card 1
    nendf = 20 # unit for endf tape
    npend = 21 # unit for pendf tape
    ngout1 = 0 # unit for input gout tape (default=0)
    ngout2 = 31 # unit for output gout tape (default=0)

    card1 = [nendf, npend, ngout1, ngout2]

    # Set Card 2
    # matb -- (already defined) -- material to be processed
    ign = 17 # neutron group structure option
    igg = 0 # gamma group structure option
    iwt = 11 # weight function option
    lord = 0 # Legendgre order
    ntemp = 1 # number of temperatures (default = 1)
    nsigz = 1 # number of sigma zeroes (default = 1)
    iprint = 1 # long print option (0/1=minimum/maximum) -- (default=1)
    ismooth = 1 # swith on/off smoother operation (1/0, default=1=on)

    card2 = [matb, ign, igg, iwt, lord, ntemp, nsigz, iprint]

    # Set Card 3
    title = f'"GROUPR Processing for {element}{Z}"'
    card3 = [title]

    # Set Card 4
    temp = 293.16 # temperature in Kelvin
    card4 = [temp]

    # Set Card 5
    sigz = 0 # sigma zero values (including infinity)
    card5 = [sigz]

    # Set Card 9
    mfd = 3 # file to be processed
    mtd = MTs # sections to be processed
    card9 = []
    for mt in MTs:
        mtname = mt_table[mt_table['MT'] == mt]['Reaction'].values[0] # description of section to be processed
        card9_line = f'{mfd} {mt} "{mtname}"'
        card9.append(card9_line)

    # Set Card 10
    matd = 0 # next mat number to be processed
    card10 = [matd]

    # Create a card deck
    deck = [card1, card2, card3, card4, card5, card9, card10]
    deck_names = ['Card 1', 'Card 2', 'Card 3', 'Card 4', 'Card 5', 'Card 9', 'Card 10']
    deck_df = pd.DataFrame({
        'Card' : deck_names,
        'Contents' : deck
    })

    # WRITE INPUT FILE FROM CARDS

    # Write the input deck to the groupr.inp file
    with open('groupr.inp', 'w') as f:
        f.write('groupr\n')
        for card_name, card_content in zip(deck_names, deck):
            f.write(format_card(card_name, card_content))
        f.write(' 0/\nstop')

    return deck_df

# Function to execute NJOY bash script
def run_njoy(endf_path, pendf_path):
    # Read the template file
    with open('run_njoy_template.sh', 'r') as f:
        script_content = f.read()

    # Replace placeholders with actual file paths
    script_content = script_content.replace('<ENDF_PATH>', endf_path)
    script_content = script_content.replace('<PENDF_PATH>', pendf_path)

    # Write modified script content to run_njoy.sh file
    with open('run_njoy.sh', 'w') as f:
        f.write(script_content)
    
    # Make the script executable
    subprocess.run(["chmod", "+x", "run_njoy.sh"])

    # Execute the modified script
    njoy_run_message = subprocess.run(["./run_njoy.sh"], capture_output=True, text=True)
    print(njoy_run_message.stdout)
    print(njoy_run_message.stderr)

    # If the run is successful, print out the output
    if njoy_run_message.stderr == '':
        output = subprocess.run(['cat', 'output'], capture_output=True, text = True)
        title = card_deck[card_deck['Card'] == 'Card 3']['Contents'].values[0][0][1:-1]
        title_index = output.stdout.find(title)
        print(output.stdout[:title_index + len(title)])


# Call TENDL download function by user CLI input
element = input('Select element: ')
Z = input('Select atomic number, Z = ')
endf_path = tendl_download(element, Z, 'endf')
pendf_path = tendl_download(element, Z, 'pendf')
print(f'ENDF file can be found at ./{endf_path}')
print(f'PENDF file can be found at ./{pendf_path}')

# Extract necessary MT and MAT data from the ENDF file
matb, MTs = endf_specs(endf_path)

# Write out the GROUPR input file
mt_table = pd.read_csv('./mt_table.csv')
card_deck = groupr_input(matb, mt_table)

# Run NJOY
run_njoy(endf_path, pendf_path)

ENDF file can be found at ./tape20
PENDF file can be found at ./tape21
Running NJOY...
NJOY ran successfully. Output written to /output.


1*****************************************************************************
 *                                         *              *                  *
 *   $$   $$       $$   $$$$$   $$    $$   *              *  vers: 2016.75   *
 *   $$$  $$       $$  $$$$$$$   $$  $$    *  nuclear     *  vday: 29Apr24   *
 *   $$$$ $$       $$  $$   $$    $$$$     *  data        *  site: lanl t2   *
 *   $$ $$$$  $$   $$  $$   $$     $$      *  processing  *  mach:           *
 *   $$  $$$  $$$$$$$  $$$$$$$     $$      *  system      *  date: 06/06/24  *
 *   $$   $$   $$$$$    $$$$$      $$      *              *  time: 15:47:53  *
 *                                         *              *                  *
 *****************************************************************************

 groupr...compute self-shielded group-averaged cross-sections         